In [1]:
import sys
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import matplotlib.patches as patches
from descartes.patch import PolygonPatch
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure

import networkx as nx
import pygraphviz
import pydot
from networkx.drawing.nx_agraph import graphviz_layout

import random
import pandas as pd
import napari

import random
from shapely.geometry import Point,MultiPoint,MultiPolygon,Polygon,box,LineString
from shapely import affinity
import re

from shapely import geometry
import pickle
import time

#import ipywidgets as widgets
#import ipywidgets.widgets.interaction
#from ipywidgets import *
import warnings
warnings.simplefilter('ignore', np.RankWarning)

from copy import deepcopy
from magicgui import magicgui
import skimage.measure as measure
from dask_image.imread import imread
#import src

from napari_animation import Animation
from napari.utils import nbscreenshot
from napari_animation import AnimationWidget

from PIL import Image

%load_ext autoreload

/Users/esb/opt/anaconda3/lib/python3.7/site-packages/napari/_qt/__init__.py:37: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [2]:
path_to_uj='../uJ/uJ_src_pBGT/python/'
sys.path.insert(0, path_to_uj)

uj_data_path="../../data/"


In [3]:
#%autoreload 2
##from readroi import *
from DataManagers import *
#from TrackingFunctions import *
#from PlottingFunctions import *
#from AnalysisFunctions import *
##from CorrectorFunctionsG import *

Importing readroi
DataStructs...loaded!
DataManagers...loaded!


In [4]:

frame2min=10

figxlim=640
figylim=512
flims=(figxlim,figylim)


channels=["DIC","GFP","DsRed"]
channels_color=["gray","green","red"]
vis=[False,True,True]
vis=[False,False,False]
seg_chan=channels[1]

max_frames=-1 #All

## Experimental parameters: MG/GT

In [5]:

rootDir=uj_data_path+"MGGT-Pulse/"

expeLabel="MGGT-AMP-Pulse"

frame_experiment_start=10;
frame_experiment_end=34
frame_signal_start=16
frame_signal_end=20
frame_norm_start=10
frame_norm_end=15                 
frame_analysis_end=34

n_frames=frame_experiment_end-frame_experiment_start+1

t_experiment_start=0
t_signal_start=(frame_signal_start-frame_experiment_start)*frame2min  
t_signal_end=(frame_signal_end-frame_experiment_start)*frame2min
t_experiment_end=(frame_experiment_end-frame_experiment_start)*frame2min

frames=range(int(frame_experiment_start),int(frame_experiment_end+1))
frames_list=[x for x in frames]
timesMG=[(this_frame-frame_experiment_start)*frame2min for this_frame in frames]
times_hour=[(this_frame-frame_experiment_start)*frame2min/60 for this_frame in frames]

seg_chan=channels[1]
mask_chan='MERGE'
data_type="normData"

full_trap_list="xy01,xy04,xy05,xy06,xy07,xy09,xy12,xy13".split(",")

trap_list=["xy09"]  #"xy01",

thres_length=47.16756886212940
thres_DsRed=218.40875

current_experiment="Chromosome"


## Experimental data

In [6]:

dirNameDataLineages=rootDir+'data/lineages/'
dirNameDataCellsTrackedCorrected=rootDir+'data_cells_tracked_corrected/'
dirNameDataLineagesCorrected=rootDir+'data/lineages_corrected/'
dirNameDataNapariLabels=rootDir+'data_napari_labels/'

trap_masks_path=[]
trap_tif_path=[]
for pos in trap_list:
    tif_path=rootDir+"data_raw/"+pos+"/"
    trap_tif_path.append(tif_path)
    masks_path=rootDir+"data_masks/"+pos+"/"+mask_chan+"/"
    trap_masks_path.append(masks_path)
    

### Load segmentation

In [7]:


trap_tracked_cells=[]
trap_tracked_cells_corrected=[]
trap_corrected_list=[]
for trap in trap_list:
    dirNameTRACKEDCELLS=rootDir+'data_cells_tracked/'+trap+"/"+data_type+"/"
    
    dirNameTRACKEDCELLS_Corrrected=dirNameDataCellsTrackedCorrected+trap+"/"+data_type+"/"
    iscorrected=os.path.exists(dirNameTRACKEDCELLS_Corrrected)
    trap_corrected_list.append(iscorrected)
    if(iscorrected):
        print("Loading corrected cells for",trap)
        tracked_cells_corrected=load_cells(dirNameTRACKEDCELLS_Corrrected,max_frames)
        trap_tracked_cells_corrected.append(tracked_cells_corrected)
    else:
        print("Loading original cells for",trap)
        tracked_cells=load_cells(dirNameTRACKEDCELLS,max_frames)
        trap_tracked_cells.append(tracked_cells)
    
        tracked_cells_corrected=tracked_cells.copy()
        trap_tracked_cells_corrected.append(tracked_cells_corrected)
    

Loading corrected cells for xy09
0: Loading MGGT-AMP-Pulse_xy09_010.pkl
1: Loading MGGT-AMP-Pulse_xy09_011.pkl
2: Loading MGGT-AMP-Pulse_xy09_012.pkl
3: Loading MGGT-AMP-Pulse_xy09_013.pkl
4: Loading MGGT-AMP-Pulse_xy09_014.pkl
5: Loading MGGT-AMP-Pulse_xy09_015.pkl
6: Loading MGGT-AMP-Pulse_xy09_016.pkl
7: Loading MGGT-AMP-Pulse_xy09_017.pkl
8: Loading MGGT-AMP-Pulse_xy09_018.pkl
9: Loading MGGT-AMP-Pulse_xy09_019.pkl
10: Loading MGGT-AMP-Pulse_xy09_020.pkl
11: Loading MGGT-AMP-Pulse_xy09_021.pkl
12: Loading MGGT-AMP-Pulse_xy09_022.pkl
13: Loading MGGT-AMP-Pulse_xy09_023.pkl
14: Loading MGGT-AMP-Pulse_xy09_024.pkl
15: Loading MGGT-AMP-Pulse_xy09_025.pkl
16: Loading MGGT-AMP-Pulse_xy09_026.pkl
17: Loading MGGT-AMP-Pulse_xy09_027.pkl
18: Loading MGGT-AMP-Pulse_xy09_028.pkl
19: Loading MGGT-AMP-Pulse_xy09_029.pkl
20: Loading MGGT-AMP-Pulse_xy09_030.pkl
21: Loading MGGT-AMP-Pulse_xy09_031.pkl
22: Loading MGGT-AMP-Pulse_xy09_032.pkl
23: Loading MGGT-AMP-Pulse_xy09_033.pkl
24: Loading MGGT-

### Load lineages files

In [8]:
trap_df_lineages_all=[]
trap_df_lineages_corrected_all=[]

for i,trap in enumerate(trap_list):
    
    if(trap_corrected_list[i]):
        print("Loading corrected lineages !")
        lineagesDataFile='%s%s_%s_lineages_filtered.csv'%(dirNameDataLineagesCorrected, expeLabel, trap)
        df_lineagesC_all=load_data_lineages(lineagesDataFile)
        trap_df_lineages_corrected_all.append(df_lineagesC_all)
    else:
        lineagesDataFile='%s%s_%s_lineages_all.csv'%(dirNameDataLineages, expeLabel, trap)
        df_lineages_all=load_data_lineages(lineagesDataFile)
        trap_df_lineages_all.append(df_lineages_all)
        print("Loading original lineages !")
        df_lineagesC_all=df_lineages_all.copy()
        trap_df_lineages_corrected_all.append(df_lineagesC_all)
#df_lineages=df_lineages_all

Loading corrected lineages !
Loading 1308 lineages from ../../data/MGGT-Pulse/data/lineages_corrected/MGGT-AMP-Pulse_xy09_lineages_filtered.csv


### Add geometric info to dataframe

In [9]:
trap_df_lineages_geoms=[]

for i,trap in enumerate(trap_list):
    if(trap_corrected_list[i]):
        tracked_cells=trap_tracked_cells_corrected[i]
        df_lineages=trap_df_lineages_corrected_all[i]
    else:
        tracked_cells=trap_tracked_cells[i]
        df_lineages=trap_df_lineages_all[i]
    trap_newDF_list=[]
    for row in df_lineages.itertuples(index=False):
        
        this_id=str(row.cellID)
        
        trackid=row.trackID
        f=this_id
        n='0'
        if this_id.count('.')>0:
            f,n=this_id.split('.')
            
        this_cell=tracked_cells[int(f)-frame_experiment_start][int(n)]
        new_df=pd.DataFrame.from_records([{"cellID":this_id,"trackID":trackid,"cellColor":this_cell['cellColor'],"center":this_cell['center'],"roiPoly":this_cell['roiPoly'],"axis":this_cell['axis']}], index='cellID')
        trap_newDF_list.append(new_df)
    trap_newDF=pd.concat(trap_newDF_list)
    trap_newDF['trackID']=trap_newDF['trackID'].astype(str)
    df_lineages['cellID']=df_lineages['cellID'].astype(str)
    df_lineages_geoms=pd.merge(df_lineages,trap_newDF,on=['cellID','trackID'])
    trap_df_lineages_geoms.append(df_lineages_geoms)
df_lineages_geoms.head(3)

,lineageID,trackID,cellID,motherID,roiID,trapID,frame,length,division,state,tracking_score,GFP,DsRed,cellColor,center,roiPoly,axis
0,1,10.095-34.000,10.095,10.095,roi_f10_n95_x167_y294,xy09,10,34.800268,1,1,17.102747,118.017,108.153,#05c564,POINT (166.9807625191048 293.9770736630306),"POLYGON ((172 295, 170 295, 170 296, 168 296, ...","LINESTRING (153.674497628185 305, 153.67852464..."
1,1,10.095-34.000,11.106,10.095,roi_f11_n106_x152_y301,xy09,11,28.419258,1,1,187.939953,118.550,108.043,#05c564,POINT (152.7414082157459 301.9009477265283),"POLYGON ((163 302, 161 302, 161 303, 158 303, ...","LINESTRING (139 304.6114237831773, 139.0503159..."
2,1,10.095-34.000,12.084,10.095,roi_f12_n84_x149_y303,xy09,12,21.311954,0,1,130.754641,118.361,107.676,#05c564,POINT (148.8423886796268 304.0288686563056),"POLYGON ((158 304, 156 304, 156 305, 153 305, ...","LINESTRING (139 306.7940798391289, 139.1121819..."


In [10]:
empty_mask=np.zeros((n_frames,figylim,figxlim))   ### Creates empty matrix array
matrix_shape = empty_mask.shape[1:]
roi_mask=deepcopy(empty_mask)

trap_roi_masks=[]
trap_cellID_to_lineage=[]
for ti,trap in enumerate(trap_list):

    print("Loading Napari files...")
    this_file=dirNameDataNapariLabels+"%s_%s_roi_mask.npy"%(trap,seg_chan)
    roi_mask=np.load(this_file)
    this_file=dirNameDataNapariLabels+"%s_%s_colors_dict.npy"%(trap,seg_chan)
    colorsCid=np.load(this_file,allow_pickle=True).item()
              
    this_file=dirNameDataNapariLabels+"%s_%s_colors_lineages_dict.npy"%(trap,seg_chan)
    colorsLid=np.load(this_file,allow_pickle=True).item()
        
    this_file=dirNameDataNapariLabels+"%s_%s_cell_lineages_dict.npy"%(trap,seg_chan)
    cellID_to_lineage=np.load(this_file,allow_pickle=True).item()
        
    this_file=dirNameDataNapariLabels+"%s_%s_cell_lineages_dict_corrected.npy"%(trap,seg_chan)
            
    trap_roi_masks.append(roi_mask)
    trap_cellID_to_lineage.append(cellID_to_lineage)
print("Done!")



Loading Napari files...
Done!


## Select cells to visualize

In [11]:


for this_trap in trap_list:

    
    ##Filtered cells:
    #trackIDs_file=('%sdata/tracks_complete_info.csv'%rootDir)
    #df = pd.read_csv(trackIDs_file)
    #df_trackIDs_filtered=df.loc[(df['experiment_id'] == current_experiment) & (df['trap_id'] == this_trap)]
    #trackIDs=df_trackIDs_filtered['track_id'].values.tolist()

  
    #All cells
    trackIDs=df_lineages_geoms.trackID.unique()  
    #trackIDs=trackIDs[50:75] #dev
  

    print(this_trap,": Selected trackIDs ",len(trackIDs),':',trackIDs)


xy09 : Selected trackIDs  1308 : ['10.095-34.000' '10.062-34.001' '10.064-34.002' ... '10.261-11.293'
 '10.272-11.299' '10.038-11.300']


In [12]:

def get_many_data_mask(tids, lbl_data):
    data_mask=deepcopy(empty_mask)
    for tid in tids:
        track_df=df_lineages_geoms[df_lineages_geoms["trackID"]==tid]
        for row in track_df.itertuples():
            this_poly=row.roiPoly
            if lbl_data is 'GFP':
                cell_data=float(row.GFP)
            elif lbl_data is 'DsRed':
                cell_data=float(row.DsRed)
            elif lbl_data is 'length':
                cell_data=float(row.length)
            elif lbl_data is 'division':
                cell_data=float(row.division)
            else:
                cname=tid.split('-')[0]
                cell_data=int(float(cname)*1e3) #TrackID
            
            cell_id=float(row.cellID)
            napari_coords=np.array([[int(y),int(x)] for (x,y) in this_poly.exterior.coords])
            mask = measure.grid_points_in_poly(matrix_shape, napari_coords)
            frame=row.frame-frame_experiment_start
            cname=tid.split('-')[0]
            data_mask[frame][mask] = cell_data
    return data_mask

In [13]:




def get_many_state_mask(tids, thres_DsRed, thres_length):
    state_mask=deepcopy(empty_mask)
    
    dead_tids=[]
    
    for tid in tids:
        track_df=df_lineages_geoms[df_lineages_geoms["trackID"]==tid]
        for row in track_df.itertuples():
            this_poly=row.roiPoly
            cell_id=float(row.cellID)
            cell_length=float(row.length)
            cell_DsRed=float(row.DsRed)
            cell_GFP=float(row.GFP)
            cell_state=float(row.state)
            
            #if tid in dead_tids:
            #    cell_state=1 #Dead
            #else:
            #    cell_state=2 #Alive
            #    if cell_DsRed>thres_DsRed:
            #        cell_state=1 #Dead
            #        if tid not in dead_tids:
            #            dead_tids.append(tid)
            #    else:
            #        if cell_length>thres_length:
            #            cell_state=3 #Elongated
            
            
            napari_coords=np.array([[int(y),int(x)] for (x,y) in this_poly.exterior.coords])
            mask = measure.grid_points_in_poly(matrix_shape, napari_coords)
            frame=row.frame-frame_experiment_start
            cname=tid.split('-')[0]
            state_mask[frame][mask] = cell_state
    return state_mask



def add_labels_state(trackIDs, this_trap, thres_DsRed, thres_length, dx, dy):
    #print("Drawing ",trackIDs)
    
    #clear_my_layers()
    #trackids=df_lineages_geoms.trackID.unique()
    #track_mask=get_many_data_mask(trackIDs, 'state')
    track_mask=get_many_state_mask(trackIDs, thres_DsRed, thres_length)
    
    color_state = {0: 'black', 1: '#629CFF', 2:'#FCF995',3:'#F8766D'}
    #track_mask=get_many_track_mask(trackIDs)
    trackLayer=cellViewer.add_labels(data=track_mask, name=this_trap+"_state", blending="additive", opacity=.9, translate=[dy, dx])
    trackLayer.color_mode = 'direct'
    trackLayer.color = color_state

    #LoadKeyBindedFuctions()
    cellViewer.window.activate()
    cellViewer.window.qt_viewer.console.execute()
   

In [14]:

def get_many_track_mask(tids):
    track_mask=deepcopy(empty_mask)
    for tid in tids:
        track_df=df_lineages_geoms[df_lineages_geoms["trackID"]==tid]
        for row in track_df.itertuples():#(index=False):
            this_poly=row.roiPoly
            cell_id=float(row.cellID)
            napari_coords=np.array([[int(y),int(x)] for (x,y) in this_poly.exterior.coords])
            mask = measure.grid_points_in_poly(matrix_shape, napari_coords)
            frame=row.frame-frame_experiment_start
            cname=tid.split('-')[0]
            int_track_mask=int(float(cname)*1e3)
            track_mask[frame][mask] = int_track_mask
    return track_mask

def add_labels_trackID(trackIDs, this_trap, dx, dy):
    #print("Drawing ",trackIDs)
    
    #clear_my_layers()
    #trackids=df_lineages_geoms.trackID.unique()
    
    track_mask=get_many_track_mask(trackIDs)
    trackLayer=cellViewer.add_labels(data=track_mask, color=colorsCid, name=this_trap+"_trackID", blending="additive", opacity=.9, translate=[dy, dx])
    trackLayer.color_mode = 'direct'
    trackLayer.color = colorsCid

    #LoadKeyBindedFuctions()
    cellViewer.window.activate()
    cellViewer.window.qt_viewer.console.execute()
    

In [15]:
def add_labels_data(trackIDs, this_trap, lbl_data, lbl_cmap, dx, dy, contrast_limits=[]):
    
    #print("Drawing",lbl_data,": ",trackIDs)
    
    data_mask=get_many_data_mask(trackIDs, lbl_data)
    
    if len(contrast_limits)==0:
        contrast_limits=[np.min(data_mask), np.max(data_mask)]
    print('contrast_limit=',contrast_limits)
        
    lbl_data='data_%s_%s'%(this_trap, lbl_data)
    
    #print(data_mask)
    #color = {-1: 'cyan', 0: 'yellow', -1: 'red'}
    #dataLayer=cellViewer.add_labels(data_mask, color=color, name=lbl_data, blending="additive", opacity=1)
    
    
    dataLayer=cellViewer.add_image(data_mask, colormap=lbl_cmap, name=lbl_data, blending="additive", opacity=1, contrast_limits=contrast_limits, translate=[dy, dx])
        
    #LoadKeyBindedFuctions()
    cellViewer.window.activate()
    cellViewer.window.qt_viewer.console.execute()
    
    

## Supplementary Figure 3

In [16]:

%gui qt

cellViewer=napari.Viewer()
cellViewer.title="Supp Movie 3"

trap_index=0
    
dx=0
dy1=-325
dy2=325

print(trap_tif_path[trap_index])

tif_path=trap_tif_path[trap_index]

#Top
cellViewer.open(tif_path+"GFP/",blending='additive',colormap="green",name=this_trap+"_GFP",visible=True,translate=[dx, dy1])
#add_labels_state(trackIDs, this_trap, thres_DsRed, thres_length, dx, dy2)        

#Bottom
cellViewer.open(tif_path+"DsRed/",blending='additive',colormap="magenta",name=this_trap+"_DsRed",visible=True,translate=[dx, dy1])
#add_labels_data(trackIDs, this_trap, 'length', 'magenta', dx, dy1)
#add_labels_data(trackIDs, this_trap, 'GFP', 'green', dx, dy1)
#cellViewer.dims.set_axis_label(0,"frame(-%s)"%frame_experiment_start)

add_labels_state(trackIDs, this_trap, thres_DsRed, thres_length, dy2, dx)        
    
cellViewer.dims.set_axis_label(0,"frame(-%s)"%frame_experiment_start)


../../data/MGGT-Pulse/data_raw/xy09/


/Users/esb/opt/anaconda3/lib/python3.7/site-packages/napari/layers/labels/labels.py:383: UserWarning: Float dtypes are not supported for Labels layers. Converting data to integers...
  "Float dtypes are not supported for Labels layers. Converting data to integers..."
Exception in callback BaseAsyncIOLoop._handle_events(87, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(87, 1)>
Traceback (most recent call last):
  File "/Users/esb/opt/anaconda3/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "/Users/esb/opt/anaconda3/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "/Users/esb/opt/anaconda3/lib/python3.7/site-packages/zmq/eventloop/zmqstream.py", line 448, in _handle_events
    self._handle_recv()
  File "/Users/esb/opt/anaconda3/lib/python3.7/site-packages/zmq/eventloop/zmqstream.py", line 477, in _handle_recv
    self._run_callback(callback, msg